In [1]:
# Import required libraries
import pandas as pd
import dash
#import dash_html_components as html
#import dash_core_components as dcc
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv'
spacex_df = pd.read_csv(URL)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Select all unique sites from the data for the entries of the dropdown box.
#print(spacex_df.head())
unique_launch_sites = spacex_df['Launch Site'].unique()
#print(type(unique_launch_sites)) 
#print(unique_launch_sites) 
option_list = [{'label': 'All Sites', 'value': 'ALL'}]
for launch_site in unique_launch_sites:
    dict_options={'label' :  launch_site  , 'value' :  launch_site }
    option_list.append(dict_options)


# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40})
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                , dcc.Dropdown( id='site-dropdown'
#                                              , options=[ {'label': 'All Sites', 'value': 'ALL'}
#                                                        , {'label': 'site1', 'value': 'site1'}
                                              , options=option_list 
                                              , value='ALL' 
                                              , placeholder="place holder here"
                                              , searchable=True
                                              )
                                , html.Br()
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                , html.Div(dcc.Graph(id='success-pie-chart'))
#                                , html.Br()
                                # Function decorator to specify function input and output
                                          
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                , html.P("Payload range (Kg):")
                                , dcc.RangeSlider( id='payload-slider'
                                                 , min=0, max=10000, step=1000
#                                                 , marks={0: '0', 100: '100'}
                                                 , value=[2000, 8000]
                                                 )

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                , html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( Output(component_id='success-pie-chart', component_property='figure')
             , Input (component_id='site-dropdown'    , component_property='value')
             )
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        filtered_df = spacex_df
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Distribution of number of launches over the launch sites')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(filtered_df,  names='class', title='Succes rate of the launches (1 = success)')
    # return the outcomes piechart for a selected site
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback( Output(component_id='success-payload-scatter-chart', component_property='figure')
             , Input (component_id='site-dropdown'                , component_property='value')
             , Input (component_id='payload-slider'               , component_property='value')
             )
def get_scatter_chart(entered_site, payload_range_selection):
    print('payload_range_selection is :', payload_range_selection)
    if entered_site == 'ALL':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range_selection[0]) & (spacex_df['Payload Mass (kg)'] <= payload_range_selection[1])]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] >= payload_range_selection[0]) & (filtered_df['Payload Mass (kg)'] <= payload_range_selection[1])]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version')
    # return the outcomes piechart for a selected site
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server()
